In [1]:
import import_ipynb #Allows you to import ipynb files (TEMPORARY?)
from queue import PriorityQueue
from dataclasses import dataclass
import math

In [2]:
#Node class that the A* will be referencing to check and compare costs
@dataclass
class AStarNode:
    
    def __init__(self, parent=None, position:tuple=None):
        self.position = position
        self.parent = parent
        self.g = 0 #distance to start node
        self.h = 0 #distance to goal node
        
        self.rsindex = -1
    def FCost(self):
        return self.g + self.h
    
    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self):
        return f"{self.position} - g: {self.g} h: {self.h} f: {self.FCost()}"
    
    def __lt__(self, other):
        return self.FCost() < other.FCost()
    
    def __gt__(self, other):
        return self.FCost() > other.FCost()

In [11]:
#This will implement the basic A* Algorithm
class AStar:
    #class variables
    def __init__(self, grid, start, end):
        self._grid = grid
        self._start = start
        self._end = end
        
    #gets all neighbors at current node
    def GetNeighborsAtPosition(self, current_node):
        neighbors = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            #out of bounds check
            if node_position[0] > (len(self._grid) - 1) or node_position[0] < 0 or node_position[1] > (len(self._grid[len(self._grid) - 1]) - 1) or node_position[1] < 0:
                continue
            #obstacle check
            if self._grid[node_position[0]][node_position[1]] != 0:
                continue
            new_node = AStarNode(current_node, node_position)
            neighbors.append(new_node)
        return neighbors
    
    #Generates the Path
    def GeneratePath(self, current):
        path = []
        while current is not None:
            path.append(current.position)
            current = current.parent
        return path[::-1]
    
    #Gets distance from one node to another
    def CalculateDistance(self, current, destination):
        return math.sqrt(((current.position[0] - destination.position[0]) ** 2 + (current.position[1] - destination.position[1]) ** 2))
    
    #Runs the algorithm
    def run(self):
        start_node = AStarNode(None, self._start)
        end_node = AStarNode(None, self._end)
        open_list = PriorityQueue()
        closed_list = []
        open_list.put(start_node)
        
        while open_list.qsize():
            current_node = open_list.get()
            closed_list.append(current_node)
            
            if(current_node == end_node):
                return self.GeneratePath(current_node)
            
            neighbors = self.GetNeighborsAtPosition(current_node)
            open_neighbors = neighbors
            for index, neighbor in enumerate(neighbors):
                for closed in closed_list:
                    if neighbor == closed:
                        open_neighbors.pop(index)
                        
            #calculate cost of each path and add to queue
            for neighbor in open_neighbors:
                neighbor.g = current_node.g + 1
                neighbor.h = self.CalculateDistance(neighbor, end_node)
                open_list.put(neighbor)         